In [7]:
import mido
import os
import re
from pathlib import Path
import json

In [3]:
dataset_folder_path = '/Users/gabriel.alvarado/Repos/thinkai/datasets/maestro-v3.0.0-midi'


folder_data = os.listdir(dataset_folder_path)
valid_folder_data = [folder for folder in folder_data if re.match(r'\d{4}', folder)]
print(valid_folder_data)

['2013', '2014', '2015', '2008', '2006', '2009', '2017', '2018', '2011', '2004']


In [4]:
# Create midi output folder if it doesn't exist
output_folder = './midi-processed'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [8]:
def process_midi_file(midi_file_path):
    midi_file = mido.MidiFile(midi_file_path)
    return json.dumps({
        "metadata": {
            "type": midi_file.type,
            "ticks_per_beat": midi_file.ticks_per_beat,
            "length": midi_file.length,
            "track_count": len(midi_file.tracks)
        },
        "messages": [str(msg) for track in midi_file.tracks for msg in track]
    })

def save_midi_data(midi_data, output_folder, filename):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(f'{output_folder}/{filename}.json', 'w') as f:
        f.write(str(midi_data))

def process_midi_folder(folder_path, output_folder):
    midi_files = os.listdir(folder_path)
    for midi_file in midi_files:
        midi_file_path = f'{folder_path}/{midi_file}'
        midi_data = process_midi_file(midi_file_path)
        save_midi_data(midi_data, output_folder, midi_file)


In [ ]:
print (valid_folder_data)

In [ ]:

for folder in valid_folder_data:
    print("processing folder: ", folder)
    folder_path = f'{dataset_folder_path}/{folder}'
    process_midi_folder(folder_path, f'{output_folder}/{folder}')

processing folder:  2013
